In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
import plotly.express as px
import emd
tf.keras.mixed_precision.set_global_policy('mixed_float16')

BATCH_SIZE = 32
CHANNEL_NUMBER = 4
WINDOW_SIZE = 300
SLIDING_STEP = int(WINDOW_SIZE * 0.4)
CLASS_NUMBER = 3
if CLASS_NUMBER < 2:
    CLASS_NUMBER = 2

def slicingAndNormalize(arr):
    def normalizer(X):
        mean = np.average(X, axis=1)
        std = np.std(X, axis=1)
        ret = (X.copy() - mean) / std
        return ret
    
    totalLength = arr.shape[0]
    if totalLength <= WINDOW_SIZE:
        return arr
    ret = normalizer((arr[:WINDOW_SIZE, :])[np.newaxis, :])
    
    i = SLIDING_STEP
    while (totalLength - i) > WINDOW_SIZE:
        new = normalizer((arr[i:(i + WINDOW_SIZE), :])[np.newaxis, :])
        ret = np.concatenate([ret, new], axis=0)
        i += SLIDING_STEP
    return ret

def slicing(x, y):
    totalLength = x.shape[0]
    assert totalLength == y.shape[0], "Data numbers not matching with that of labels."
    if totalLength <= WINDOW_SIZE:
        return x
    
    halfWindow = 0.5 * WINDOW_SIZE
    retx = (x[:WINDOW_SIZE, :])[np.newaxis, :]
    rety = [1] if np.sum(y[:WINDOW_SIZE]) > halfWindow else [0]
    
    i = SLIDING_STEP
    while (totalLength - i) > WINDOW_SIZE:
        new = (x[i:(i + WINDOW_SIZE), :])[np.newaxis, :]
        retx = np.concatenate([retx, new], axis=0)
        rety.append(1) if np.sum(y[i:(i + WINDOW_SIZE)]) > halfWindow else rety.append(0)
        i += SLIDING_STEP
    return retx, np.array(rety)

def emdSignal(sig):
    dataNumber = sig.shape[0]
    channel = sig.shape[-1]
    ret = None
    
    for i in range(dataNumber):
        temp = None
        
        for c in range(channel):
            raw = sig[i, :, c]
            imf = emd.sift.sift(raw, max_imfs=4, imf_opts={'sd_thresh': 0.1})
            
            if imf.shape[-1] < 4:
                compensate = np.zeros((WINDOW_SIZE, 4 - imf.shape[-1]))
                imf = np.concatenate([imf, compensate], axis = 1)
            
            if not type(temp) == np.ndarray: 
                temp = imf
            else: 
                temp = np.concatenate([temp, imf], axis = 1)
            
        if type(temp) == np.ndarray: 
            if not type(ret) == np.ndarray: 
                ret = temp[np.newaxis, :]
            else: 
                ret = np.concatenate([ret, temp[np.newaxis, :]], axis = 0)
                
    return ret

def buildModel(shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape = shape),
        tf.keras.layers.Conv1D(80, 40, padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(axis=1),
        tf.keras.layers.MaxPool1D(padding='same'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Conv1D(100, 50, padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(axis=1),
        tf.keras.layers.MaxPool1D(padding='same'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Conv1D(120, 60, padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(axis=1),
        tf.keras.layers.MaxPool1D(padding='same'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(CLASS_NUMBER, activation='softmax')]
    )
    model.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.CategoricalAccuracy(),
                         tf.keras.metrics.Precision(thresholds = 0.5),
                         tf.keras.metrics.Recall(thresholds= 0.5)])
    model.summary()
    return model

In [6]:
trainFile = './data/lick.npy'
trainLabel = "./data/lick_y.npy"
X, y = slicing(np.load(trainFile), np.load(trainLabel))
X = emdSignal(X)
print(X.shape)

(81, 300, 12)


In [ ]:
threeChan = X[18]
f = threeChan[:, 0].T
s = threeChan[:, 1].T
t = threeChan[:, 2].T
imf = emd.sift.sift(f, max_imfs=4, imf_opts={'sd_thresh': 0.1})
print(imf.shape)
emd.plotting.plot_imfs(imf)

In [ ]:
n += 1
print(n, n*SLIDING_STEP, n*SLIDING_STEP + WINDOW_SIZE)
px.line(X[n]).show()

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=777)

In [16]:
model = buildModel(X.shape[1:])
history = model.fit(x=X_train,
                    y=y_train,
                    epochs=100,
                    validation_data=(X_test,y_test))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_6 (Conv1D)           (None, 300, 80)           38480     
                                                                 
 batch_normalization_6 (Batc  (None, 300, 80)          1200      
 hNormalization)                                                 
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 150, 80)          0         
 1D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 150, 80)           0         
                                                                 
 conv1d_7 (Conv1D)           (None, 150, 100)          400100    
                                                                 
 batch_normalization_7 (Batc  (None, 150, 100)        

c:\Python310\lib\site-packages\keras\backend.py:5673: UserWarning:

"`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?



2/2 [==============================] - 19s 479ms/step - loss: 0.7814 - categorical_accuracy: 1.0000 - precision_2: 0.6364 - recall_2: 0.9722 - val_loss: 23.7500 - val_categorical_accuracy: 1.0000 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00
Epoch 2/100
2/2 [==============================] - 0s 50ms/step - loss: 2.3562 - categorical_accuracy: 1.0000 - precision_2: 0.6735 - recall_2: 0.9167 - val_loss: 34.3438 - val_categorical_accuracy: 1.0000 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00
Epoch 3/100
2/2 [==============================] - 8s 8s/step - loss: 0.5635 - categorical_accuracy: 1.0000 - precision_2: 0.6857 - recall_2: 0.6667 - val_loss: 72.5625 - val_categorical_accuracy: 1.0000 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00
Epoch 4/100
2/2 [==============================] - 0s 44ms/step - loss: 1.3228 - categorical_accuracy: 1.0000 - precision_2: 0.6889 - recall_2: 0.8611 - val_loss: 31.4062 - val_categorical_accuracy: 1.0000 - val_precision_2

In [3]:
model = tf.keras.models.load_model('./model/LickingPark')

In [9]:
model.predict(X_test)
X_test.shape[0]

1/1 [==============================] - 0s 21ms/step


21

In [18]:
model.save("LickenPark", save_format="tf")

In [12]:
f1 = np.array(history.history['loss']).flatten()
valf1 = np.array(history.history['val_loss']).flatten()
px.line(pd.DataFrame(np.array([f1, valf1]).T, columns=['loss', 'val_loss'])).show()